In [2]:
# Import Functions
import pandas as pd
import os
import time
import csv
import json
import fnmatch
import csv
import sqlalchemy as sql
import numpy as np
dbHost='mxnunx41'
dbPort='1521'
dbUser='lims'
dbPassword='Mumwit63'
dbName='hwi1'
dburl='oracle://'

In [3]:
# Default Parameters setting
with open('.\configFiles\HachConfig.json', 'r') as f:
    config = json.load(f)
lab = config['lab']
analysis = config['analysis']
inst = config['inst']
colsOrder=['Sample.Sample_Number', 'Test.Test_Number','Result.Name', 'Result.Entry', 'Result.Entered_By']

In [4]:
# get Data from DB
def getDataFromDB(sampleNumbersList):
    dburl = 'oracle://'+dbUser+':'+dbPassword+'@'+dbHost+':'+dbPort+'/'+dbName
    engine=sql.create_engine(dburl)
    resultdf=pd.DataFrame()
    sampleNumbrStr=(', '.join("'" + item + "'" for item in sampleNumbersList))
    queryText="SELECT sample_number, analysis, REPLICATE_COUNT,TEST_NUMBER FROM test"
    queryText=queryText+" WHERE SAMPLE_NUMBER IN ("+sampleNumbrStr+") AND analysis IN ('SODIUM-ISE')"
    conn = engine.connect()
    resultdf = pd.read_sql(queryText, engine)
    conn.close()
    return resultdf

In [5]:
def formatSampleNumber(sampleNumstring):
    return sampleNumstring[:9]

In [6]:
#write the file
def writeFile(dframe,lab,inst,archDestination,importDest,datapath,fname):
    timestr = time.strftime("%Y%m%d%H%M%S")
    FileName = importDest + lab+'_'+inst+'_' + timestr + '.csv'
    dframe.to_csv(FileName, index=False, encoding="utf-8",quotechar='"',quoting=csv.QUOTE_NONNUMERIC) # write  out the file to be imported
    destination = archDestination + fname[:-4] + '.bak' # Backup the processed file
    os.rename(datapath, destination)
    time.sleep(1)

In [6]:
for eachlab in lab:
    path='//mxns.loc//shares//NA-Instruments//Prod//'+eachlab+'//Import//'+inst+'//'
    archivedDestination='//mxns.loc//shares//NA-Instruments//Prod//'+eachlab+'//Import//'+inst+'//'
    importDest='//mxns.loc//shares//NA-Instruments//Prod//'+eachlab+'//impf//'
    print(path)
    if os.path.exists(path):
        break
    else:
        os.makedirs(path)
    files = os.listdir(path)
    for filename in files:
        if fnmatch.fnmatch(filename,'*.CSV'):
            print (filename)

Lab:Allentown
//mxns.loc//shares//NA-Instruments//Prod//Allentown//Import//Hach//


In [8]:
path='//mxnws073//LimsImport//LimsImportFiles//HACH//ALN//ARC//'

In [9]:
filename='HQ40dUpload_20190410_160241.csv'
datapathxl = path + str(filename)
rawdf = pd.read_csv(datapathxl,encoding='utf-8')

In [10]:
rawdf

,Type,Parameter Type,Date,Time,Operator ID,Probe Model,Probe SN,Method Name,Sample ID,Primary Reading Value,...,Cal Std 3 Supp Value,Cal Std 4,Cal Std 4 Units,Cal Std 4 Primary Value,Cal Std 4 Primary Units,Cal Std 4 Supp Value,Cal Std 5,Cal Std 5 Units,Cal Std 5 Primary Value,Cal Std 5.1
0,RD,Na,10-4-2019,1:20:06 PM,- - -,ISENa381,<183402907692,Default,378076457 (009),686,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RD,Na,10-4-2019,1:22:55 PM,- - -,ISENa381,<183402907692,Default,378076457 (010),316,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Type,Parameter Type,Date,Time,Operator ID,Probe Model,Probe SN,Method Name,Sample ID,Primary Reading Value,...,Cal Std 3 Supp Value,Cal Std 4,Cal Std 4 Units,Cal Std 4 Primary Value,Cal Std 4 Primary Units,Cal Std 4 Supp Value,Cal Std 5,Cal Std 5 Units,Cal Std 5 Primary Value,Cal Std 5
3,RD,Na,10-4-2019,1:29:43 PM,- - -,ISENa381,<183402907692,Default,387024614 (001),317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RD,Na,10-4-2019,1:33:12 PM,- - -,ISENa381,<183402907692,Default,387114711 (001),685,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
ImpCols=['Type','Sample ID','Primary Reading Value']

In [13]:
rawdf=rawdf[ImpCols]

In [14]:
rawdf

,Type,Sample ID,Primary Reading Value
0,RD,378076457 (009),686
1,RD,378076457 (010),316
2,Type,Sample ID,Primary Reading Value
3,RD,387024614 (001),317
4,RD,387114711 (001),685


In [15]:
rawdf.drop(rawdf[rawdf.Type !='RD'].index, inplace=True)

c:\users\agundlapalli\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [16]:
rawdf['sample_number']=rawdf['Sample ID'].apply(formatSampleNumber)

c:\users\agundlapalli\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
rawdf

,Type,Sample ID,Primary Reading Value,sample_number
0,RD,378076457 (009),686,378076457
1,RD,378076457 (010),316,378076457
3,RD,387024614 (001),317,387024614
4,RD,387114711 (001),685,387114711


In [19]:
sampleNumbersList=list(rawdf['sample_number'].unique())

In [20]:
databasedf=getDataFromDB(sampleNumbersList)

In [21]:
rawdf=rawdf.astype({"sample_number": int})

In [22]:
dfResult = rawdf.merge(databasedf, how = 'inner', on = ['sample_number'])

In [23]:
dfResult=dfResult.rename(index=str, 
columns={'sample_number': 'Sample.Sample_Number', 'resultname': 'Result.Name','Primary Reading Value': 'Result.Entry','test_number':'Test.Test_Number'})

In [24]:
dfResultReplicateMax = dfResult.groupby('Sample.Sample_Number')['replicate_count'].max().reset_index()

In [25]:
dfResultReplicateMax

,Sample.Sample_Number,replicate_count
0,378076457,3
1,387024614,1
2,387114711,3


In [26]:
dfResult

,Type,Sample ID,Result.Entry,Sample.Sample_Number,analysis,replicate_count,Test.Test_Number
0,RD,378076457 (009),686,378076457,SODIUM-ISE,1,631112923
1,RD,378076457 (009),686,378076457,SODIUM-ISE,2,631112924
2,RD,378076457 (009),686,378076457,SODIUM-ISE,3,631112925
3,RD,378076457 (010),316,378076457,SODIUM-ISE,1,631112923
4,RD,378076457 (010),316,378076457,SODIUM-ISE,2,631112924
5,RD,378076457 (010),316,378076457,SODIUM-ISE,3,631112925
6,RD,387024614 (001),317,387024614,SODIUM-ISE,1,645525470
7,RD,387114711 (001),685,387114711,SODIUM-ISE,2,645741553
8,RD,387114711 (001),685,387114711,SODIUM-ISE,3,645741556
9,RD,387114711 (001),685,387114711,SODIUM-ISE,1,645736074


In [19]:
dfResultFinal = dfResult.merge(dfResultReplicateMax, how = 'inner', on = ['Sample.Sample_Number','replicate_count'])

In [20]:
dfResultFinal['Result.Name']='Inst. Read-out'

In [21]:
dfResultFinal['Result.Entered_By']='IMPORTER'

In [22]:
dfResultFinal = dfResultFinal[colsOrder]

In [23]:
dfResultFinal

,Sample.Sample_Number,Test.Test_Number,Result.Name,Result.Entry,Result.Entered_By
0,378076457,631112925,Inst. Read-out,686,IMPORTER
1,378076457,631112925,Inst. Read-out,316,IMPORTER
2,387024614,645525470,Inst. Read-out,317,IMPORTER
3,387114711,645741556,Inst. Read-out,685,IMPORTER


In [1]:
writeFile(dfResultFinal,lab,inst,path,importDest,datapath,fname)

NameError: name 'writeFile' is not defined